## About

In this notebook we prepare a simple solution for the [kaggle challenge on trigger system.](https://inclass.kaggle.com/c/trigger-system)

In [1]:
import pandas
import numpy
from rep.utils import train_test_split_group
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

### Download data

In [2]:
!cd datasets; wget -O training.csv -nc --no-check-certificate https://2016.mlhep.yandex.net/data/training.csv

--2016-06-20 10:31:34--  https://2016.mlhep.yandex.net/data/training.csv
Resolving 2016.mlhep.yandex.net... 2a02:6b8::1:208, 213.180.193.208
Connecting to 2016.mlhep.yandex.net|2a02:6b8::1:208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 83646329 (80M) [application/octet-stream]
Saving to: 'training.csv'

training.csv        100%[===================>]  79.77M  1.26MB/s    in 1m 40s  

2016-06-20 10:33:15 (816 KB/s) - 'training.csv' saved [83646329/83646329]



In [3]:
!cd datasets; wget -O test.csv -nc --no-check-certificate https://2016.mlhep.yandex.net/data/test.csv

--2016-06-20 10:33:17--  https://2016.mlhep.yandex.net/data/test.csv
Resolving 2016.mlhep.yandex.net... 2a02:6b8::1:208, 213.180.193.208
Connecting to 2016.mlhep.yandex.net|2a02:6b8::1:208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68944694 (66M) [application/octet-stream]
Saving to: 'test.csv'

test.csv            100%[===================>]  65.75M  1.44MB/s    in 44s     

2016-06-20 10:34:02 (1.49 MB/s) - 'test.csv' saved [68944694/68944694]



### Read training and test files

In [4]:
data = pandas.read_csv('datasets/training.csv')
test = pandas.read_csv('datasets/test.csv')

In [5]:
data.head()

,EventID,Label,Mass,Corrected_mass,Pt,Pt_sum,Pt_min,IP_chi2,IP_chi2_sum,Flight_distance,Pseudorapidity,Track_number_PV,Tracks_number,Tracks_number_passed,Vertex_chi2,Weight
0,0,1,2461.369995,3958.329995,5521.529995,5553.419995,2351.519995,116.829995,1457.219995,3004.729995,3.267565,0,2,2,0.000095,2.828167
1,1,1,3361.719998,3853.869998,4022.719998,4362.719998,936.834998,1.512528,259.576998,289.585998,3.461578,0,2,1,1.151478,2.828167
2,1,1,790.218991,5305.539991,4305.589991,4309.419991,891.455991,190.981991,270.212991,252.402991,3.612741,0,2,1,0.179642,2.828167
3,1,1,2631.380000,11106.300000,3147.820000,3840.460000,419.138000,82.668400,243.689000,242.731000,3.459510,1,2,1,2.147980,2.828167
4,1,1,4343.369999,6410.599999,3811.409999,4777.559999,421.900999,6.995589,263.454999,266.133999,3.851009,1,3,1,8.621519,2.828167


### Define training features

Exclude `EventID`, `Label` and `Weight` from the features set

In [6]:
features = list(set(data.columns) - {'EventID', 'Label', 'Weight'})
features

['IP_chi2_sum',
 'Flight_distance',
 'Pt',
 'Tracks_number',
 'Pt_sum',
 'Corrected_mass',
 'Track_number_PV',
 'Pseudorapidity',
 'Tracks_number_passed',
 'Mass',
 'IP_chi2',
 'Pt_min',
 'Vertex_chi2']

### Divide training data into 2 parts
Here `train_test_split_group` function is used to divide into 2 parts to preserve secondary vertices from the same events in the same part of data (training or test). First argument should be events ids.

In [7]:
training_data, validation_data = train_test_split_group(data.EventID, data, random_state=11, train_size=0.66)

### Simple gradient boosting from `sklearn` training

We take all secondary vertices (SVs) for all events and train on them.

In [8]:
gb = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, subsample=0.8, random_state=13,
                                min_samples_leaf=100, max_depth=6, max_features=8)
gb.fit(training_data[features], training_data.Label)

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=6, max_features=8, max_leaf_nodes=None,
              min_samples_leaf=100, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=13, subsample=0.8, verbose=0,
              warm_start=False)

### Prepare predictions, labels and weights for events (not for SVs!) on the cross-validation sample

In [9]:
def compute_mean(event_ids, values):
    """ fore each event computes average of given values """
    number_of_sv_in_event = numpy.bincount(event_ids)
    return numpy.bincount(event_ids, weights=values) / number_of_sv_in_event

In [10]:
# Example of usage
compute_mean(event_ids=[0, 1, 3, 1], values=[1, 2, 3, 4])

array([  1.,   3.,  nan,   3.])

In [11]:
# predict each SV
proba = gb.predict_proba(validation_data[features])
events_ids = numpy.unique(validation_data.EventID)

# compute number of SVs in each event
number_of_sv_in_event = numpy.bincount(validation_data.EventID)

# compute predictions for events (take the mean value of predictions for SVs forming an event)
events_proba = compute_mean(validation_data.EventID, proba[:, 1])[events_ids]

# compute weights for events 
events_weights = compute_mean(validation_data.EventID, validation_data.Weight)[events_ids]

# compute labels for events 
events_labels = compute_mean(validation_data.EventID, validation_data.Label)[events_ids]

### ROC AUC for events (with weights) on the cross validation sample

In [12]:
roc_auc_score(events_labels, events_proba, sample_weight=events_weights)

0.94717430152820903

## Prepare submission to kaggle

In [13]:
# predict each SV in test sample
kaggle_proba = gb.predict_proba(test[features])

kaggle_ids = numpy.unique(test.EventID)
# compute predictions for events (take the mean value of predictions for SVs forming an event)
kaggle_events_proba = compute_mean(test.EventID, kaggle_proba[:, 1])[kaggle_ids]

In [14]:
from IPython.display import FileLink
def create_solution(ids, proba, filename='baseline.csv'):
    """saves predictions to file and provides a link for downloading """
    pandas.DataFrame({'EventID': ids, 'Label': proba}).to_csv('datasets/{}'.format(filename), index=False)
    return FileLink('datasets/{}'.format(filename))
    
create_solution(kaggle_ids, kaggle_events_proba)

/Users/antares/Yandex.Disk.localized/projects/MLHEP/mlhep2016/kaggle_triggers/datasets/baseline.csv